In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

import easyocr
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm

In [2]:
# Initialize EasyOCR reader (outside the loop)
reader = easyocr.Reader(['en'], gpu=True)

# Function to extract text using EasyOCR
def extract_text_from_image(image_path):
    try:
        result = reader.readtext(image_path, detail=0)
        result_string = ' '.join(result)  # Join the OCR results into a single string
        return result_string
    except Exception as e:
        print(f"Error extracting text from {image_path}: {e}")
        return None


In [3]:
image_path = "https://datapartner.btpr.online/ProductPictures/_WhatsAppImage2024-07-24at17.09.24_fb9bde05.jpg"
text = extract_text_from_image(image_path)
print(text)

Progress: |████████████████████████████████████████████████████████| 112.9% CompleteTRANSACTION ID -100004468004419 Thank YouHarnit snan null;We nave also #ent tne Ddoking dalailsto ttttttshhaEail com +91-722+T+*099 KARNAVATi (129341 PNR:O- Ubtubyn 09.29 Zh AAlm- 12.10 Mon 02 Oct Kon; 02 Oct Vapi(vapii Mumbal Central(MMc 2AuL 0 Chila GENERAL Mapl (VA Pr Oc 2023 Book RETURN JOURNEY Canccinndn Rctnnenunn Passenger Information HARSHIT SHAH Male| 30yrs Ceerd CNF/D8/48NWS CNF/D8/48/WS LABDHI MODI Frmalc| 24"C 08456bR CNF/D8/49NWS CNF/D8/49/WS You can now book Hotei Retiring Room Bus Order Pantry Order Food Unatain 'dinegut alsaIte Jeiggy dineout 40% OFF AFLORE How was your booking experience? Payment details 8Jse Fare 200 Convenience i2e 11,80 Tocol 2125 Intormation Mcttue MerecduemDnc Talu averofe Thia Licket i  booked petona Lter Ityralel purchzse Oftence Ws che Rallvtavy AcC 198] For enqulry and integrated rlvay helpllne ie75 dlal Please visit"My booklngs' vew Your Uckels QR Code EXP  Fo

In [4]:
from transformers import AutoModel, AutoTokenizer
import torch
torch.cuda.empty_cache()  # Clears GPU cache memory

In [ ]:
model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True, torch_dtype=torch.half)
model = model.half().to(device='cuda:0') 
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True)

# Define the function to process each text input
def process_text_input(extracted_text, prompt,image_path):
    try:
        # Prepare the input text in the required format for the model
        input_text = f"Extracted text: {extracted_text}\nPrompt: {prompt}"

        # Use the input format that matches the model's expected input structure
        msgs = [{'role': 'user', 'content': input_text}]

        with torch.no_grad():
            res = model.chat(
                image=image_path,
                msgs=msgs,
                tokenizer=tokenizer,
                sampling=True,
                temperature=0.7,
                stream=False
            )

        # Collect the generated response
        generated_text = "".join(res)  # Concatenate the generated responses

        return generated_text.strip()  # Return the final output
    except Exception as e:
        print(f"Error processing text: {e}")
        return ""  # Return an empty string if there's an error


ticket_details_output_format = "PNR No : <digit> /n Passenger Name : <name>"
prompt_template = f"Identify the informations from the extraceted text and give in the fomat{ticket_details_output_format}. answer intelligentaly"

In [6]:
result = process_text_input(text,prompt_template,image_path)

In [7]:
result

tensor([[[-3.6013e-01, -1.9337e-01, -7.9374e-02,  ..., -8.8492e-03,
           3.8162e-01,  3.3828e-01],
         [-9.9554e-01,  7.4537e-02,  1.0738e-02,  ..., -3.0701e-01,
           1.5694e-02,  1.4877e-01],
         [-5.1554e-01, -1.8726e-01,  5.1874e-01,  ..., -2.0678e-01,
           2.0827e-01,  3.2034e-01],
         ...,
         [-3.9155e-01, -7.7073e-02,  2.5944e-01,  ..., -4.8605e-01,
          -2.8372e-01,  3.2239e-02],
         [ 1.0052e-01, -4.4140e-01,  3.0597e-01,  ...,  8.9071e-04,
          -3.0852e-02,  3.1440e-02],
         [ 8.2484e-01,  9.8327e-02, -6.9836e-02,  ...,  3.2548e-01,
          -3.7559e-01, -3.1392e-01]]], device='cuda:0')